# Data preparation

The aim of this notebook is to read the grid of simulations and prepare files ready to be used by the training code. This should avoid the computation time which was spent at the beginning of each training reading and deprojecting the simulation results. 
Furthermore, in this way we separate the preprocessing of the data from the training code allowing to reuse the latter more easily.


## 1. Load the parameters file

In [4]:
import pandas as pd
import radiative_transfer as rt
para = pd.read_csv('../../../../../tesi/main/data/datarun4/downf/labelled_para.csv', index_col=0)
#para = pd.read_csv('params.csv', index_col=0)

In [5]:
para

,InvStokes1,Alpha,AspectRatio,FlaringIndex,PlanetMass,SigmaSlope,nx,rout,ny,contrast500,contrast1000,contrast1500
0,377.572191,0.008299,0.031715,0.258475,0.000139,1.01695,991.0,3,319.0,0.665589,0.684977,0.694446
1,28.379190,0.000147,0.077285,0.099575,0.000060,0.69915,600.0,3,193.0,0.266067,0.331192,0.402497
2,205.589060,0.000154,0.057615,0.165025,0.000559,0.83005,600.0,3,193.0,0.996393,0.997001,0.999980
3,601.173737,0.000509,0.096115,0.334775,0.002195,1.16955,600.0,3,193.0,0.989254,0.998590,0.999877
4,209.411246,0.002636,0.047675,0.131075,0.000451,0.76215,659.0,3,212.0,0.991362,0.995581,0.995757
...,...,...,...,...,...,...,...,...,...,...,...,...
995,250.610925,0.000138,0.078405,0.130025,0.007612,0.76005,600.0,5,242.0,0.999474,1.000000,NaN
996,15.595525,0.000526,0.042635,0.142275,0.000015,0.78455,737.0,3,237.0,0.374074,0.430613,0.453701
997,131.522483,0.000249,0.046695,0.148225,0.000014,0.79645,673.0,3,217.0,0.160138,0.249747,0.317984
998,342.767787,0.000330,0.070565,0.336175,0.000674,1.17235,600.0,3,193.0,0.984655,0.993640,0.999550


## 2. Select the time of the desired dumps

In [6]:
orbits_time = 500
label_time = int(orbits_time/50)

## 3. Load all the images projecting to cartesian coordinates and remove missing lines from the parameters' table

In [7]:
import oofargo
import numpy as np
from tqdm import tqdm 
import os
import astropy.units as u
data_path = '../../../../../tesi/main/data/datarun4/downf/'


In [8]:
def load_data():
    return np.array([[i, 
                  oofargo.open_img(
                      f'{data_path}out_{i:05}/dust1dens{label_time}.dat',
                      ntheta = para.loc[i, 'nx'].astype(int),
                      nr = para.loc[i, 'ny'].astype(int),
                      image_rmax = para.loc[i, 'rout'],
                      ylog = True
                  )]
                for i in tqdm(para.index.tolist())
                 if os.path.exists(f'{data_path}out_{i:05}/dust1dens30.dat')
                ])
    
def load_data_rt():
    return [[i, 
                  np.array(rt.radiative_transfer(
                      f'{data_path}out_{i:05}/dust1dens{label_time}.dat',
                      para, 
                      i
                  )/u.K)]
                for i in tqdm(para.index.tolist())
                 if os.path.exists(f'{data_path}out_{i:05}/dust1dens{label_time}.dat')
                ]

In [9]:
data = np.array(load_data_rt(), dtype=object)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:43<00:00, 22.78it/s]


## 5. Remove images without a visible gap

In [10]:
import cv2

# return the maximum value for the log derivative between 0.6-2.4
def getmaxlogder(image, nx, ny, rmin, rmax, deproject=False):
    if isinstance(image, str):
        rho = oofargo.open_img(image, nr=ny, ntheta=nx, image_rmin=rmin, image_rmax=rmax, ylog=True)
    elif deproject:
        rho = cv2.warpPolar(image, (ny,nx), (64,64), 64, cv2.WARP_FILL_OUTLIERS )
    else:
        rho = image
    prof = np.log10(rho.mean(axis=0))
    y = np.log10(np.linspace(rmin, rmax, ny))
    
    der = (prof[1:]-prof[:-1])/(y[1:]-y[:-1])
    
    #remove part of the profile
    der_r = der[int((0.6-rmin)*ny/(rmax-rmin)):int((2.4-rmin)*ny/(rmax-rmin))]
    maxder = der_r.max()
    minder = der_r.min()
    
    return maxder

In [11]:
# select objects with gaps: the selection is done on the data at time 1500
from scipy.ndimage import gaussian_filter
if orbits_time == 1500:
    para_n = para.loc[data[:,0]]
    for i, img in data:
        para_n.loc[i, f'maxder{orbits_time}'] = getmaxlogder(gaussian_filter(img, 2).transpose(), 
                                           para_n.loc[i, 'nx'].astype(int),
                                           para_n.loc[i, 'ny'].astype(int),
                                           0.4, para_n.loc[i, 'rout'], deproject=False)
    selected = para_n[para_n[f'maxder{orbits_time}']>0].sort_values(f'maxder{orbits_time}')
    para_n.to_csv('params.csv')
    selected.to_csv('selected.csv')
else:
    selected = pd.read_csv('selected.csv', index_col=0)

In [12]:
#remove objects without gaps
inputs = np.array([np.array([i, img], dtype=object) for i, img in data if i in selected.index.tolist()], dtype=object)

## Augmentation 

In [13]:
# function used to extrapolate the density outside of the outer radius
def augment(image, nx, ny, rmin, rmax, rtarget, slope):
    
    #create new grid of r
    r = np.arange(rmin, rtarget, (rmax-rmin)/ny)
    new_ny = len(r)
    
    #extrapolate profile
    if new_ny < ny:
        return image[:new_ny, :], new_ny
    else:
        padded_im = np.pad(image, ((0, new_ny-ny),(0,0)),'constant', constant_values=(0,))
        rgrid = np.ones((new_ny, nx))*r.reshape(-1,1)
        prof = image[-1, :]*(rgrid/rmax)**(-slope)*(np.arange(0,new_ny,1)>ny-1).astype(int).reshape(-1,1)
        return prof+padded_im, new_ny

In [14]:
from scipy.ndimage import gaussian_filter
# function used to augment and warp the images
# Note: warp must be done before of the downsampling in order to avoid artefacts
def augment_and_warp(image, rtarg, nx, ny, rmin, rmax, slope, smooth=3):
    im_data, new_ny = augment(image, nx, ny, rmin, rmax, rtarg, slope)
    
    img =  oofargo.warp_image_rolltodisk(im_data, nx, new_ny, image_rmax = rtarg, target_rmax=4, target_image_size=(1280,1280))
    normalized = (img-img.mean())/(img.std())
    #norm_noisy = np.array(GaussianNoise(0.1*normalized.max())(normalized, True))
    #--> with gaussian filter
        #img = gaussian_filter(cv2.resize(normalized, (128,128), interpolation=cv2.INTER_AREA), 2)  
    #--> without gaussian filter
    img = cv2.resize(normalized, (128,128), interpolation=cv2.INTER_AREA)
    #imglog = img.copy()*(img>0.01).astype(int) + (img<=0.01).astype(int)*0.01
    #imglog = (np.log10(imglog)+2)/2
    return img

In [15]:
selected

,InvStokes1,Alpha,AspectRatio,FlaringIndex,PlanetMass,SigmaSlope,nx,rout,ny,contrast500,contrast1000,contrast1500,maxder1500
627,881.048873,0.001435,0.093595,0.299775,0.000296,1.09955,600.0,3,193.0,0.225999,0.291627,0.320794,0.006546
309,126.765187,0.000114,0.054115,0.054425,0.000013,0.60885,600.0,3,193.0,0.084523,0.146130,0.172629,0.026093
761,26.001596,0.002223,0.099265,0.104475,0.000278,0.70895,600.0,3,193.0,0.319072,0.346207,0.339264,0.030646
601,89.742879,0.000104,0.097375,0.091525,0.000087,0.68305,600.0,3,193.0,0.208117,0.245891,0.264464,0.041852
223,43.551187,0.000452,0.076655,0.032375,0.000063,0.56475,600.0,3,193.0,0.243220,0.255303,0.254995,0.108617
...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,11.297959,0.000122,0.092895,0.175525,0.000427,0.85105,600.0,3,193.0,1.000000,1.000000,1.000000,319.310724
762,10.115795,0.000299,0.035285,0.305725,0.000795,1.11145,890.0,3,286.0,1.000000,1.000000,1.000000,325.825071
400,11.040786,0.000146,0.031435,0.023975,0.000028,0.54795,999.0,3,321.0,1.000000,1.000000,1.000000,332.931263
320,13.396767,0.000259,0.036685,0.138075,0.000894,0.77615,856.0,3,276.0,1.000000,1.000000,1.000000,335.583967


In [16]:
params = ['Alpha','AspectRatio','InvStokes1','FlaringIndex','PlanetMass','SigmaSlope']
params_n = ['Alpha_n','AspectRatio_n','InvStokes1_n','FlaringIndex_n','PlanetMass_n','SigmaSlope_n']
max = np.array([1e-2, 0.1, 1e3, 0.35, 1e-2, 1.2])
min = np.array([1e-4, 0.03, 10, 0, 1e-5, 0.5])
    
for i,pa in enumerate(params):
    #['Alpha','AspectRatio','InvStokes1','FlaringIndex','PlanetMass','SigmaSlope']
    if pa in ['Alpha', 'InvStokes1', 'PlanetMass']:
        max[i] = np.log10(max[i])
        min[i] = np.log10(min[i])
        selected[f'{pa}_n'] = np.log10(selected[pa])
    else:
        selected[f'{pa}_n'] = selected[pa]
        
    selected[f'{pa}_n'] = 2*(selected[f'{pa}_n']-min[i])/(max[i]-min[i]) -1

In [17]:
selected

,InvStokes1,Alpha,AspectRatio,FlaringIndex,PlanetMass,SigmaSlope,nx,rout,ny,contrast500,contrast1000,contrast1500,maxder1500,Alpha_n,AspectRatio_n,InvStokes1_n,FlaringIndex_n,PlanetMass_n,SigmaSlope_n
627,881.048873,0.001435,0.093595,0.299775,0.000296,1.09955,600.0,3,193.0,0.225999,0.291627,0.320794,0.006546,0.157,0.817,0.945,0.713,-0.019,0.713
309,126.765187,0.000114,0.054115,0.054425,0.000013,0.60885,600.0,3,193.0,0.084523,0.146130,0.172629,0.026093,-0.945,-0.311,0.103,-0.689,-0.921,-0.689
761,26.001596,0.002223,0.099265,0.104475,0.000278,0.70895,600.0,3,193.0,0.319072,0.346207,0.339264,0.030646,0.347,0.979,-0.585,-0.403,-0.037,-0.403
601,89.742879,0.000104,0.097375,0.091525,0.000087,0.68305,600.0,3,193.0,0.208117,0.245891,0.264464,0.041852,-0.983,0.925,-0.047,-0.477,-0.373,-0.477
223,43.551187,0.000452,0.076655,0.032375,0.000063,0.56475,600.0,3,193.0,0.243220,0.255303,0.254995,0.108617,-0.345,0.333,-0.361,-0.815,-0.467,-0.815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,11.297959,0.000122,0.092895,0.175525,0.000427,0.85105,600.0,3,193.0,1.000000,1.000000,1.000000,319.310724,-0.915,0.797,-0.947,0.003,0.087,0.003
762,10.115795,0.000299,0.035285,0.305725,0.000795,1.11145,890.0,3,286.0,1.000000,1.000000,1.000000,325.825071,-0.525,-0.849,-0.995,0.747,0.267,0.747
400,11.040786,0.000146,0.031435,0.023975,0.000028,0.54795,999.0,3,321.0,1.000000,1.000000,1.000000,332.931263,-0.835,-0.959,-0.957,-0.863,-0.707,-0.863
320,13.396767,0.000259,0.036685,0.138075,0.000894,0.77615,856.0,3,276.0,1.000000,1.000000,1.000000,335.583967,-0.587,-0.809,-0.873,-0.211,0.301,-0.211


In [18]:
params_n

['Alpha_n',
 'AspectRatio_n',
 'InvStokes1_n',
 'FlaringIndex_n',
 'PlanetMass_n',
 'SigmaSlope_n']

In [19]:
# split in 5 folds
from sklearn.model_selection import KFold


num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=74)

splitted_data = {}
fold = 1
for train, test in kfold.split(inputs):
    for r_i in range(3):
        
        inputs_tr = np.array([
            augment_and_warp(img, 
                            rtarg = np.random.uniform(2.5+r_i*0.5, 3+r_i*0.5),
                            nx = selected.loc[i, 'nx'].astype(int),
                            ny = selected.loc[i, 'ny'].astype(int),
                            rmin = 0.4,
                            rmax = selected.loc[i, 'rout'],
                            slope = selected.loc[i, 'SigmaSlope']
                            ) for i, img in tqdm(inputs[train])])
        if r_i == 0:
            inputs_aug = inputs_tr
        else:
            inputs_aug = np.concatenate([inputs_aug, inputs_tr], axis=0)
        
    inputs_te = np.array([
            augment_and_warp(img, 
                            rtarg = np.random.uniform(2.5,4),
                            nx = selected.loc[i, 'nx'].astype(int),
                            ny = selected.loc[i, 'ny'].astype(int),
                            rmin = 0.4,
                            rmax = selected.loc[i, 'rout'],
                            slope = selected.loc[i, 'SigmaSlope']
                            ) for i, img in tqdm(inputs[test])])
    splitted_data[f'inp_train{fold}'] = inputs_aug
    splitted_data[f'inp_test{fold}'] = inputs_te
    splitted_data[f'targ_train{fold}'] = np.tile(np.array((selected.loc[inputs[train, 0], params_n])), 3)
    splitted_data[f'targ_test{fold}'] = np.array(selected.loc[inputs[test, 0], params_n])
    fold+=1

100%|██████████| 143/143 [00:04<00:00, 33.04it/s]


## 6. Save files
In an appropriate folder inside `training_data` there will be these files:

In [20]:
import os
folder = f'training_data/only_subs_nosmooth_{orbits_time}/'
os.mkdir(folder)
np.save(f'{folder}/data', splitted_data)